In [1]:
import os
os.environ["OMP_NUM_THREADS"]='1'
import osmnx as ox
import networkx as nx
from datetime import datetime

import pandas as pd
import geopandas as gpd

from pyproj import Transformer 
import numpy as np
import scipy
from descartes import PolygonPatch

import matplotlib.pyplot as plt
from matplotlib import *
from matplotlib.patches import *
from matplotlib.lines import Line2D
import matplotlib.colors as mcolors

import contextily as cx #Copyright (c) 2016, Dani Arribas-Bel. All rights reserved.

import math
import itertools
from shapely.geometry import MultiPolygon, Polygon, Point, LineString
from shapely.ops import nearest_points
from IPython.display import Image, IFrame

from sklearn import cluster
from sklearn.cluster import KMeans


import pandapower as pp
import pandapower.timeseries as ts
import pandapower.control as control
from pandapower.plotting import simple_plot, simple_plotly, pf_res_plotly 
import pandapower.plotting.plotly as plotly


In [2]:
def delete_all_components(net):
    # Alle Busse löschen
    net.bus.drop(net.bus.index, inplace=True)
    
    # Alle Erzeugungen löschen
    net.gen.drop(net.gen.index, inplace=True)
    net.sgen.drop(net.sgen.index, inplace=True)
    net.storage.drop(net.storage.index, inplace=True)

    # Alle Lasten löschen
    net.load.drop(net.load.index, inplace=True)
    
    # Alle Leitungen löschen
    net.line.drop(net.line.index, inplace=True)
    
    # Alle externen Netze löschen
    net.ext_grid.drop(net.ext_grid.index, inplace=True)

In [3]:
def initialize_network():
    # Erstellen eines neuen leeren Netzwerks
    return pp.from_json('Daten/Netzmodell_PandaPower_Weilimdorf_V2.json')

In [4]:
def extract_results(net):
    # Maximale Belastung der Kabel (in %)
    max_line_loading = net.res_line.loading_percent.max()
    
    # Maximale Belastung der Trafos (in %)
    if len(net.trafo) > 0:
        max_trafo_loading = net.res_trafo.loading_percent.max()
    else:
        max_trafo_loading = 0  # Falls keine Trafos im Netz sind

    # Gesamte Erzeugungsleistung (in MW)
    total_generation = net.res_gen.p_mw.sum() + net.res_sgen.p_mw.sum() + net.res_storage.p_mw.sum()
    
    # Gesamte Last (in MW)
    total_load = net.res_load.p_mw.sum()
    
    # Weitere interessante Parameter
    max_bus_voltage = net.res_bus.vm_pu.max()
    min_bus_voltage = net.res_bus.vm_pu.min()
    total_losses = net.res_line.pl_mw.sum() + net.res_trafo.pl_mw.sum()

    return {
        "max_line_loading": max_line_loading,
        "max_trafo_loading": max_trafo_loading,
        "total_generation": total_generation,
        "total_load": total_load,
        "max_bus_voltage": max_bus_voltage,
        "min_bus_voltage": min_bus_voltage,
        "total_losses": total_losses
    }

In [5]:
def find_rings(net):
    visited_lines = set()
    rings = []

    for line_idx in net.line.index:
        if line_idx in visited_lines:
            continue
        
        # Fange einen neuen Ring an
        stack = [line_idx]
        ring_lines = set()
        ring_buses = set()

        while stack:
            current_line = stack.pop()
            
            if current_line in visited_lines:
                continue
            
            visited_lines.add(current_line)
            ring_lines.add(current_line)
            
            # Füge die Busse der aktuellen Leitung hinzu
            from_bus = net.line.at[current_line, 'from_bus']
            to_bus = net.line.at[current_line, 'to_bus']
            ring_buses.update([from_bus, to_bus])

            # Finde benachbarte Leitungen (die zu den aktuellen Bussen führen)
            connected_lines = net.line[((net.line['from_bus'] == from_bus) | 
                                        (net.line['to_bus'] == from_bus) |
                                        (net.line['from_bus'] == to_bus) | 
                                        (net.line['to_bus'] == to_bus))].index
            
            # Füge alle benachbarten Leitungen, die noch nicht besucht wurden, zum Stack hinzu
            for next_line in connected_lines:
                if next_line not in visited_lines:
                    stack.append(next_line)
        
        if ring_lines:
            rings.append((ring_buses, ring_lines))

    return rings

In [6]:
def export_network_parameters(net):
    # Erstelle eine Liste, um die Parameter zu speichern
    data = []

    # Gesamtanzahl der Busse
    total_buses = len(net.bus)
    data.append(['total_buses', total_buses])
    
    # Anzahl der Verbraucher
    total_loads = len(net.load)
    data.append(['total_loads', total_loads])

    # Anzahl der Transformatoren (Ortsnetzstationen)
    total_transformers = len(net.trafo)
    data.append(['total_transformers', total_transformers])
    
    # Gesamtanzahl der Leitungen
    total_lines = len(net.line)
    data.append(['Anzahl Leitungen', total_lines])

    # Gesamtlänge aller Leitungen
    total_line_length_km = net.line['length_km'].sum()
    data.append(['Gesamtlänge Leitungen (km)', total_line_length_km])
    
    # Finde alle Ringe im Netz
    rings = find_rings(net)

    # Anzahl der erkannten Ringe
    total_rings = len(rings)
    data.append(['Anzahl Ringe', total_rings])
    data.append(['Durchschnittliche Leitungslänge pro Ring', total_line_length_km/total_rings])

    # Optional: Details über jeden Ring
    for i, (buses, lines) in enumerate(rings):
        data.append([f'Leitungslänge Ring {i} (km)', net.line.loc[list(lines), 'length_km'].sum()])
    
    # Konvertiere die Liste in ein DataFrame
    df_parameters = pd.DataFrame(data, columns=['Parameter', 'Value'])
    
    return df_parameters

In [7]:
net = initialize_network()
param = export_network_parameters(net)
delete_all_components(net)
param

,Parameter,Value
0,total_buses,380.000000
1,total_loads,75.000000
2,total_transformers,2.000000
3,Anzahl Leitungen,294.000000
4,Gesamtlänge Leitungen (km),21.986507
5,Anzahl Ringe,6.000000
6,Durchschnittliche Leitungslänge pro Ring,3.664418
7,Leitungslänge Ring 0 (km),2.869000
8,Leitungslänge Ring 1 (km),2.060699
9,Leitungslänge Ring 2 (km),6.505334


In [8]:

def lade_messdaten(net, P):
    #Wird nur einmal benötigt

    # Erstelle einen leeren DataFrame, der später gefüllt wird
    df = pd.DataFrame()
    
    # Durchlaufe alle Busse im Netzwerk
    for bus in net.bus.name:
        # Konvertiere den Bus-Index in einen String, falls es sich um eine Zahl handelt
        bus_str = str(bus)

        # Extrahiere die Nummer des Busses aus dem Namen (z.B. "AbA 3300" -> "3300")
        bus_nummer = ''.join(filter(str.isdigit, bus_str))
        if len(bus_nummer) != 4:
            continue
        # Suche nach dem Ordner, der zu dieser Bus-Nummer gehört
        messordner = None
        for ordner in os.listdir('./Messungen'):
            ordnerpfad = os.path.join('./Messungen', ordner)

            if os.path.isdir(ordnerpfad) and bus_nummer in ordner:  # Überprüfen, ob es sich um einen Ordner handelt
                messordner = ordnerpfad
                break

        # Falls kein passender Ordner gefunden wurde, überspringe diesen Bus
        if not messordner:
            continue
        
        # Durchsuche den gefundenen Ordner nach der Datei, die mit der Busnummer beginnt
        for datei in os.listdir(messordner):
            if datei.startswith(bus_nummer) and datei.endswith(P + '.xlsx'):
                # Lade die Excel-Datei
                dateipfad = os.path.join(messordner, datei)
                df_temp = pd.read_excel(dateipfad, usecols=[2], header=0, names=[bus])
                
                # Füge die Daten dem DataFrame hinzu
                if df.empty:
                    df = df_temp
                else:
                    df = pd.concat([df, df_temp], axis=1)
                break
    
    return df

In [9]:
def lade_messdaten_normal(net,type):
    if type == "P+":
        return pd.read_excel('./Messungen/p_messungen_1.xlsx')
    elif type == "Q1":
        return pd.read_excel('./Messungen/q_messungen_1.xlsx')
    elif type == "P-":
        return pd.read_excel('./Messungen/p_gen_messungen_1.xlsx')
    else:
        print("Fehler")

In [10]:
#net = initialize_network()

result_df_p = lade_messdaten_normal(net,"P+")
result_df_q = lade_messdaten_normal(net,"Q1")
result_df_p_gen = lade_messdaten_normal(net,"P-")
#result_df_p.to_excel('./Messungen/p_messungen_1.xlsx')
#result_df_q.to_excel('./Messungen/q_messungen_1.xlsx')
#result_df_p_gen.to_excel('./Messungen/p_gen_messungen_1.xlsx')

#delete_all_components(net)


In [37]:
Last_Sammelschiene = pd.read_csv("./Messungen/WIDRF_2022_P.csv", sep=';',decimal=",", encoding='utf-8')

Spalten = ["K17","K27","K09","K29","K18","K26"]

Last_Sammelschiene_Winter = Last_Sammelschiene.iloc[1537:2209][Spalten]
Last_Sammelschiene_Fruehling = Last_Sammelschiene.iloc[10273:10273+672][Spalten]
Last_Sammelschiene_Sommer = Last_Sammelschiene.iloc[21025:21025+672][Spalten]

Last_Ring_2_Winter = -Last_Sammelschiene_Winter["K17"] - Last_Sammelschiene_Winter["K27"]
Last_Ring_1_Winter = -Last_Sammelschiene_Winter["K09"] - Last_Sammelschiene_Winter["K29"]
Last_Ring_3_Winter = -Last_Sammelschiene_Winter["K18"] - Last_Sammelschiene_Winter["K26"]

Last_Ring_2_Sommer = -Last_Sammelschiene_Sommer["K17"] - Last_Sammelschiene_Sommer["K27"]
Last_Ring_1_Sommer = -Last_Sammelschiene_Sommer["K09"] - Last_Sammelschiene_Sommer["K29"]
Last_Ring_3_Sommer = -Last_Sammelschiene_Sommer["K18"] - Last_Sammelschiene_Sommer["K26"]

Last_Ring_2_Fruehling = -Last_Sammelschiene_Fruehling["K17"] - Last_Sammelschiene_Fruehling["K27"]
Last_Ring_1_Fruehling = -Last_Sammelschiene_Fruehling["K09"] - Last_Sammelschiene_Fruehling["K29"]
Last_Ring_3_Fruehling = -Last_Sammelschiene_Fruehling["K18"] - Last_Sammelschiene_Fruehling["K26"]

In [36]:
messdaten_p_Winter = result_df_p[1537:2209]
messdaten_p_gen_Winter = result_df_p_gen[1537:2209]
messdaten_q_Winter = result_df_q[1537:2209]

messdaten_p_Sommer = result_df_p[21025:21025+672]
messdaten_p_gen_Sommer = result_df_p_gen[21025:21025+672]
messdaten_q_Sommer = result_df_q[21025:21025+672]

messdaten_p_Fruehling = result_df_p[10273:10273+672]
messdaten_p_gen_Fruehling = result_df_p_gen[10273:10273+672]
messdaten_q_Fruehling = result_df_q[10273:10273+672]


In [18]:
net2 = initialize_network()
def find__connected_buses(net):
    # 1. Löschen von Leitungen mit in_service = False
    net.line.drop(net.line[net.line.in_service == False].index, inplace=True)

    # Set zum Speichern der besuchten Busse
    visited_buses = set()
    # Liste zum Speichern der Ringe
    rings = []

    # 2. Tiefensuche (DFS) ab einem Bus, um alle verbundenen Busse zu finden
    def dfs(bus, current_ring):
        if bus in visited_buses:
            return
        visited_buses.add(bus)
        current_ring.append(bus)  # Füge den aktuellen Bus zum aktuellen Ring hinzu
        #print(f"Besuchter Bus: {bus}")

        # Finde alle verbundenen Busse über Leitungen
        for _, line in net.line.iterrows():
            if line.in_service:
                if line.from_bus == bus and line.to_bus not in visited_buses:
                    dfs(line.to_bus, current_ring)
                elif line.to_bus == bus and line.from_bus not in visited_buses:
                    dfs(line.from_bus, current_ring)

    # 3. Finde die erste in Betrieb befindliche Leitung und starte die Suche
    for _, line in net.line.iterrows():
        if line.in_service:
            start_bus = line.from_bus
            #print(f"Starte Suche ab erstem Bus der ersten Leitung: {start_bus}")
            current_ring = []  # Erstelle einen neuen Ring für die aktuelle Suche
            dfs(start_bus, current_ring)
            rings.append(current_ring)  # Speichere den aktuellen Ring
            break  # Nach der ersten Leitung aufhören und mit den restlichen Bussen fortfahren

    # 4. Starte neue Suchen für jeden nicht besuchten Bus
    for bus in net.bus.index:
        if bus not in visited_buses:
            #print(f"\nStarte neue Suche ab Bus: {bus}")
            current_ring = []  # Erstelle einen neuen Ring für die aktuelle Suche
            dfs(bus, current_ring)
            rings.append(current_ring)  # Speichere den aktuellen Ring
    # 5. Entferne Ringe mit nur einem Element
    rings = [ring for ring in rings if len(ring) > 1]
    return rings


In [19]:
def update_loads(net, load_p, load_q, generation,Last_Ring_1,Last_Ring_2,Last_Ring_3):
    # Alle Lasten löschen
    net.load.drop(net.load.index, inplace=True)
    net.sgen.drop(net.sgen.index, inplace=True)

    if isinstance(load_p, pd.Series):
        load_df_p = load_p.to_frame()
    else:
        load_df_p = load_p

    if isinstance(load_q, pd.Series):
        load_df_q = load_q.to_frame()
    else:
        load_df_q = load_q

    if isinstance(generation, pd.Series):
        gen_df = generation.to_frame()
    else:
        gen_df = generation
    
    for bus in net.bus.name:
        bus_name = str(bus)
        if bus_name in load_df_p.index:
            try:
                # Werte für P und Q holen
                P = load_df_p.loc[bus_name].iloc[0]
                if bus_name in load_df_q.index:
                    Q = load_df_q.loc[bus_name].iloc[0] 
                else:
                    Q=0

                if bus_name in gen_df.index:
                    gen = gen_df.loc[bus_name].iloc[0]
                    pp.create_sgen(net,bus_idx,p_mw = gen, name = bus_name)


                # Den Bus-Index im Pandapower-Netzwerk finden
                bus_idx = net.bus.index[net.bus['name'] == bus_name].tolist()
                if bus_idx:
                    bus_idx = bus_idx[0]  # Den ersten (und wahrscheinlich einzigen) Bus-Index verwenden
                    pp.create_load(net, bus_idx, p_mw=P, q_mvar=Q, name = bus_name)
              
            except KeyError:
                print(f"Spalte für '{bus_name}' nicht gefunden.")
    
    rings = find__connected_buses(net)

    lasten_summe = []  # Liste für die Lastsummen
    rings_copy = rings.copy()  # Eine Kopie der Liste erstellen, um beim Löschen zu vermeiden, die Original-Liste zu verändern

    lasten_summe = []  # Liste für die Lastsummen
    rings_copy = rings.copy()  # Kopie der Liste erstellen, um beim Löschen in der Original-Liste zu arbeiten
    i = 0
    # Iteration über die Kopie der Liste der Ringe
    for ring in rings_copy:
        ring_last_sum = 0
        
        for bus in ring:
            # Summiere die Lasten für jeden Bus im Ring
            ring_last_sum += net.load.loc[net.load.bus == bus, 'p_mw'].sum()  # Leistung in MW

        # Füge die berechnete Lastsumme der Liste hinzu
        lasten_summe.append(ring_last_sum)

        # Überprüfen, ob die Lastsumme null ist
        if i == 0:
            ring_last_sum = 0
            i = 1
        if ring_last_sum == 0:
            # Lösche den Ring und die zugehörige Lastsumme
            index = rings.index(ring)  
            rings.pop(index) 
            lasten_summe.pop(index)


    restlast = [Last_Ring_1 - lasten_summe[0],Last_Ring_2 - lasten_summe[1],Last_Ring_3 - lasten_summe[2]]

    i = 0
    for ring in rings:
        tmp_bus = []
        tmp_busname=[]
        for bus in ring:
            bus_name = net.bus.loc[bus, 'name']
            if bus_name.startswith("USt"):
                tmp_bus.append(bus)
                tmp_busname.append(bus_name)
        factor = 1/len(tmp_bus)
        for bus_index in tmp_bus:
            index = tmp_bus.index(bus_index)
            pp.create_load(net,bus_index, p_mw = restlast[i]*factor,name = tmp_busname[index])
        i = i + 1

    

In [20]:
results = []
plot = False
P_Zeitreihe = messdaten_p_Winter      
Q_Zeitreihe = messdaten_q_Winter
Gen_Zeitreihe = messdaten_p_gen_Winter


for j in range(len(P_Zeitreihe)):
    p = P_Zeitreihe.iloc[j]/1000
    q = Q_Zeitreihe.iloc[j]/1000
    gen = Gen_Zeitreihe.iloc[j]/1000
    p1 = Last_Ring_1_Winter.iloc[j]
    p2 = Last_Ring_2_Winter.iloc[j]
    p3 = Last_Ring_3_Winter.iloc[j]
    
    if j%1 == 0:
        print(f"Iteration {j}/{len(P_Zeitreihe)}")
    
    # Netzwerk initialisieren
    if j == 0:
        net1 = initialize_network()
    
    update_loads(net1,p,q,gen,p1,p2,p3)
    
    # Power Flow Berechnung durchführen
    pp.runpp(net1, algorithm='nr', error_tolerance_vm_pu=1e-08, symmetric=True, validate_input=False)

    # Ergebnisse speichern 
    result_data = extract_results(net1)
    result_data.update({
        "Zeitschritt": j,
    })
    results.append(result_data)

    if j%48 == 0 and plot:
        pp.plotting.pf_res_plotly(net1)    
    
    #Ergebnisse speichern
    if j == len(P_Zeitreihe) - 1:
        Jahreszeit = "Winter"
        current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        file_name_net = f"./results/{Jahreszeit}Weilimdorf_{current_time}_pp_net.xlsx"
        file_name_result = f"./results/{Jahreszeit}Weilimdorf_{current_time}_pp_net_result.xlsx"
        df = pd.DataFrame(results)
        pp.to_excel(net1, file_name_net)
        df.to_excel(file_name_result)
        break  # Abbrechen der Schleife vor der letzten Iteration
    

Iteration 0/672
Iteration 1/672
Iteration 2/672
Iteration 3/672
Iteration 4/672
Iteration 5/672
Iteration 6/672
Iteration 7/672
Iteration 8/672
Iteration 9/672
Iteration 10/672
Iteration 11/672
Iteration 12/672
Iteration 13/672
Iteration 14/672
Iteration 15/672
Iteration 16/672
Iteration 17/672
Iteration 18/672
Iteration 19/672
Iteration 20/672
Iteration 21/672
Iteration 22/672
Iteration 23/672
Iteration 24/672
Iteration 25/672
Iteration 26/672
Iteration 27/672
Iteration 28/672
Iteration 29/672
Iteration 30/672
Iteration 31/672
Iteration 32/672
Iteration 33/672
Iteration 34/672
Iteration 35/672
Iteration 36/672
Iteration 37/672
Iteration 38/672
Iteration 39/672
Iteration 40/672
Iteration 41/672
Iteration 42/672
Iteration 43/672
Iteration 44/672
Iteration 45/672
Iteration 46/672
Iteration 47/672
Iteration 48/672
Iteration 49/672
Iteration 50/672
Iteration 51/672
Iteration 52/672
Iteration 53/672
Iteration 54/672
Iteration 55/672
Iteration 56/672
Iteration 57/672
Iteration 58/672
Iterati

In [38]:
results = []
plot = False
messdaten_p_Sommer.fillna(0, inplace=True)
messdaten_q_Sommer.fillna(0, inplace=True)
messdaten_p_gen_Sommer.fillna(0, inplace=True)
P_Zeitreihe = messdaten_p_Sommer
Q_Zeitreihe = messdaten_q_Sommer
Gen_Zeitreihe = messdaten_p_gen_Sommer


for j in range(len(P_Zeitreihe)):
    p = P_Zeitreihe.iloc[j]/1000
    q = Q_Zeitreihe.iloc[j]/1000
    gen = Gen_Zeitreihe.iloc[j]/1000
    p1 = Last_Ring_1_Sommer.iloc[j]
    p2 = Last_Ring_2_Sommer.iloc[j]
    p3 = Last_Ring_3_Sommer.iloc[j]
    
    if j%1 == 0:
        print(f"Iteration {j}/{len(P_Zeitreihe)}")
    
    # Netzwerk initialisieren
    if j == 0:
        net1 = initialize_network()
    
    update_loads(net1,p,q,gen,p1,p2,p3)
    
    # Power Flow Berechnung durchführen
    pp.runpp(net1, algorithm='nr', error_tolerance_vm_pu=1e-08, symmetric=True, validate_input=False)

    # Ergebnisse speichern 
    result_data = extract_results(net1)
    result_data.update({
        "Zeitschritt": j,
    })
    results.append(result_data)

    if j%48 == 0 and plot:
        pp.plotting.pf_res_plotly(net1)    
    
    #Ergebnisse speichern
    if j == len(P_Zeitreihe) - 1:
        Jahreszeit = "Sommer_"
        current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        file_name_net = f"./results/{Jahreszeit}Weilimdorf_{current_time}_pp_net.xlsx"
        file_name_result = f"./results/{Jahreszeit}Weilimdorf_{current_time}_pp_net_result.xlsx"
        df = pd.DataFrame(results)
        pp.to_excel(net1, file_name_net)
        df.to_excel(file_name_result)
        break  # Abbrechen der Schleife vor der letzten Iteration
    

Iteration 0/672
Iteration 1/672
Iteration 2/672
Iteration 3/672
Iteration 4/672
Iteration 5/672
Iteration 6/672
Iteration 7/672
Iteration 8/672
Iteration 9/672
Iteration 10/672
Iteration 11/672
Iteration 12/672
Iteration 13/672
Iteration 14/672
Iteration 15/672
Iteration 16/672
Iteration 17/672
Iteration 18/672
Iteration 19/672
Iteration 20/672
Iteration 21/672
Iteration 22/672
Iteration 23/672
Iteration 24/672
Iteration 25/672
Iteration 26/672
Iteration 27/672
Iteration 28/672
Iteration 29/672
Iteration 30/672
Iteration 31/672
Iteration 32/672
Iteration 33/672
Iteration 34/672
Iteration 35/672
Iteration 36/672
Iteration 37/672
Iteration 38/672
Iteration 39/672
Iteration 40/672
Iteration 41/672
Iteration 42/672
Iteration 43/672
Iteration 44/672
Iteration 45/672
Iteration 46/672
Iteration 47/672
Iteration 48/672
Iteration 49/672
Iteration 50/672
Iteration 51/672
Iteration 52/672
Iteration 53/672
Iteration 54/672
Iteration 55/672
Iteration 56/672
Iteration 57/672
Iteration 58/672
Iterati

In [35]:
results = []
plot = False
messdaten_p_Fruehling.fillna(0, inplace=True)
messdaten_q_Fruehling.fillna(0, inplace=True)
messdaten_p_gen_Fruehling.fillna(0, inplace=True)
P_Zeitreihe = messdaten_p_Fruehling
Q_Zeitreihe = messdaten_q_Fruehling
Gen_Zeitreihe = messdaten_p_gen_Fruehling


for j in range(len(P_Zeitreihe)):
    p = P_Zeitreihe.iloc[j]/1000
    q = Q_Zeitreihe.iloc[j]/1000
    gen = Gen_Zeitreihe.iloc[j]/1000
    p1 = Last_Ring_1_Fruehling.iloc[j]
    p2 = Last_Ring_2_Fruehling.iloc[j]
    p3 = Last_Ring_3_Fruehling.iloc[j]

    if j%1 == 0:
        print(f"Iteration {j}/{len(P_Zeitreihe)}")
    
    # Netzwerk initialisieren
    if j == 0:
        net1 = initialize_network()
    
    update_loads(net1,p,q,gen,p1,p2,p3)
    
    # Power Flow Berechnung durchführen
    pp.runpp(net1, algorithm='nr', error_tolerance_vm_pu=1e-08, symmetric=True, validate_input=False)

    # Ergebnisse speichern 
    result_data = extract_results(net1)
    result_data.update({
        "Zeitschritt": j,
    })
    results.append(result_data)

    if j%48 == 0 and plot:
        pp.plotting.pf_res_plotly(net1)    
    
    #Ergebnisse speichern
    if j == len(P_Zeitreihe) - 1:
        Jahreszeit = "Fruehling_"
        current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        file_name_net = f"./results/{Jahreszeit}Weilimdorf_{current_time}_pp_net.xlsx"
        file_name_result = f"./results/{Jahreszeit}Weilimdorf_{current_time}_pp_net_result.xlsx"
        df = pd.DataFrame(results)
        pp.to_excel(net1, file_name_net)
        df.to_excel(file_name_result)
        break  # Abbrechen der Schleife vor der letzten Iteration
    

0.5820000000000001
0.584
2.095
11.681784
Iteration 0/672
0.581
0.587
1.962
11.694412000000005
Iteration 1/672
0.597
0.577
1.875
11.684607999999999
Iteration 2/672
0.6559999999999999
0.526
1.867
11.767687999999998
Iteration 3/672
0.696
0.532
1.902
11.721116
Iteration 4/672
0.635
0.547
2.237
11.741331999999996
Iteration 5/672
0.647
0.58
2.3520000000000003
11.742736000000006
Iteration 6/672
0.657
0.564
2.3810000000000002
11.773224
Iteration 7/672
0.6759999999999999
0.584
2.35
11.796707999999997
Iteration 8/672
0.71
0.565
2.364
11.767648
Iteration 9/672
0.6819999999999999
0.563
2.46
11.731779999999992
Iteration 10/672
0.663
0.549
2.559
11.742959999999997
Iteration 11/672
0.6739999999999999
0.567
2.5349999999999997
11.790131999999996
Iteration 12/672
0.698
0.57
2.568
11.706328
Iteration 13/672
0.673
0.553
2.4459999999999997
11.763548000000005
Iteration 14/672
0.6739999999999999
0.59
1.774
11.803420000000006
Iteration 15/672
0.706
0.58
1.556
11.748792000000002
Iteration 16/672
0.688
0.55
1.5